In [1]:
import sklearn
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.ensemble import RandomTreesEmbedding, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, SGDRegressor, RidgeCV, Ridge
from sklearn.model_selection import ParameterGrid, ParameterSampler, RandomizedSearchCV
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, KernelCenterer, RobustScaler
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer, max_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import joblib
from sklearn.model_selection import cross_val_score, KFold, cross_validate, cross_val_predict
from sklearn.pipeline import make_pipeline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, time
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, max_error
from sklearn.linear_model import LinearRegression

## BEFORE PRUNING

### -20 sampai 20

In [3]:
df_all_before = pd.read_csv('z_score_3_data_all.csv')
df_all = df_all_before[(df_all_before["α"] >= -20) & (df_all_before["α"] <= 20)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [4]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999998817175 
 Beta : 0.9999999992439083 
 V : 0.9999999852883807 
MAE : 
 Alpha : 4.033386845778907e-05 
 Beta : 8.66441816618533e-05 
 V : 0.0008175209351838255 
MSE : 
 Alpha : 1.857010364922185e-08 
 Beta : 1.1458502108277778e-07 
 V : 2.8986761131130687e-06 
Max AERR : 
 V    0.013139
α    0.001993
β    0.003830
dtype: float64
Waktu yang dibutuhkan :  0.6634801944096883


### -30 sampai 30

In [5]:
df_all_before = pd.read_csv('z_score_3_data_all.csv')
df_all = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 30)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [6]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999810457064541 
 Beta : 0.9999999999139602 
 V : 0.9999999999757284 
MAE : 
 Alpha : 0.03384909215801525 
 Beta : 4.0079046264989656e-05 
 V : 4.34074279524564e-05 
MSE : 
 Alpha : 0.00589288599522088 
 Beta : 1.348663991346294e-08 
 V : 5.080248231332654e-09 
Max AERR : 
 V    0.000980
α    0.675979
β    0.001131
dtype: float64
Waktu yang dibutuhkan :  0.9780231753985087


### -30 sampai 45

In [8]:
df_all_before = pd.read_csv('z_score_3_data_all.csv')
df_all = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 45)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [9]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999514563468 
 Beta : 0.9999449434981681 
 V : 0.9999999291782752 
MAE : 
 Alpha : 0.0021734080984078512 
 Beta : 0.006343640934383464 
 V : 0.0005478444842918906 
MSE : 
 Alpha : 2.590865027821258e-05 
 Beta : 0.006999296078625185 
 V : 1.4175266989298195e-05 
Max AERR : 
 V    0.065913
α    0.049536
β    2.502888
dtype: float64
Waktu yang dibutuhkan :  1.2530886292457581


### 30 sampai 60

In [10]:
df_all_before = pd.read_csv('z_score_3_data_all.csv')
df_all = df_all_before[(df_all_before["α"] >= 30) & (df_all_before["α"] <= 60)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [11]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999950059599891 
 Beta : 0.9996514125392504 
 V : 0.9999999888091691 
MAE : 
 Alpha : 0.013104326745178127 
 Beta : 0.05834738293605209 
 V : 0.0004258715961855142 
MSE : 
 Alpha : 0.0004434699726421998 
 Beta : 0.016531705859262103 
 V : 2.2635093098042677e-06 
Max AERR : 
 V    0.016516
α    0.129339
β    1.411254
dtype: float64
Waktu yang dibutuhkan :  0.6071378866831462


### 60 sampai 90

In [12]:
df_all_before = pd.read_csv('z_score_3_data_all.csv')
df_all = df_all_before[(df_all_before["α"] >= 60) & (df_all_before["α"] <= 90)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [13]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999986599548435 
 Beta : 0.8169764392248944 
 V : 0.9999990416052246 
MAE : 
 Alpha : 0.007311273579196406 
 Beta : 1.6264335006268689 
 V : 0.006508720287406276 
MSE : 
 Alpha : 0.00013070116760134713 
 Beta : 8.437759669203341 
 V : 0.0001910551521091634 
Max AERR : 
 V     0.074925
α     0.090241
β    15.858013
dtype: float64
Waktu yang dibutuhkan :  0.6226406097412109


## AFTER PRUNING

### -20 sampai 20

In [14]:
df_all_before = pd.read_csv('data_clean_reduce_21_z_score_3_sec.csv')
df_all = df_all_before[(df_all_before["α"] >= -20) & (df_all_before["α"] <= 20)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [15]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999998817175 
 Beta : 0.9999999992439083 
 V : 0.9999999852883807 
MAE : 
 Alpha : 4.033386845778907e-05 
 Beta : 8.66441816618533e-05 
 V : 0.0008175209351838255 
MSE : 
 Alpha : 1.857010364922185e-08 
 Beta : 1.1458502108277778e-07 
 V : 2.8986761131130687e-06 
Max AERR : 
 V    0.013139
α    0.001993
β    0.003830
dtype: float64
Waktu yang dibutuhkan :  0.6916830499966939


### -30 sampai 30

In [16]:
df_all_before = pd.read_csv('data_clean_reduce_21_z_score_3_sec.csv')
df_all = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 30)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [17]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999810457064541 
 Beta : 0.9999999999139602 
 V : 0.9999999999757284 
MAE : 
 Alpha : 0.03384909215801525 
 Beta : 4.0079046264989656e-05 
 V : 4.34074279524564e-05 
MSE : 
 Alpha : 0.00589288599522088 
 Beta : 1.348663991346294e-08 
 V : 5.080248231332654e-09 
Max AERR : 
 V    0.000980
α    0.675979
β    0.001131
dtype: float64
Waktu yang dibutuhkan :  1.0092074394226074


### -30 sampai 45

In [18]:
df_all_before = pd.read_csv('data_clean_reduce_21_z_score_3_sec.csv')
df_all = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 45)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [19]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999514563468 
 Beta : 0.9999449434981681 
 V : 0.9999999291782752 
MAE : 
 Alpha : 0.0021734080984078512 
 Beta : 0.006343640934383464 
 V : 0.0005478444842918906 
MSE : 
 Alpha : 2.590865027821258e-05 
 Beta : 0.006999296078625185 
 V : 1.4175266989298195e-05 
Max AERR : 
 V    0.065913
α    0.049536
β    2.502888
dtype: float64
Waktu yang dibutuhkan :  1.27061417500178


### 30 sampai 60

In [20]:
df_all_before = pd.read_csv('data_clean_reduce_21_z_score_3_sec.csv')
df_all = df_all_before[(df_all_before["α"] >= 30) & (df_all_before["α"] <= 60)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [21]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999950059599891 
 Beta : 0.9996514125392504 
 V : 0.9999999888091691 
MAE : 
 Alpha : 0.013104326745178127 
 Beta : 0.05834738293605209 
 V : 0.0004258715961855142 
MSE : 
 Alpha : 0.0004434699726421998 
 Beta : 0.016531705859262103 
 V : 2.2635093098042677e-06 
Max AERR : 
 V    0.016516
α    0.129339
β    1.411254
dtype: float64
Waktu yang dibutuhkan :  0.6448524435361226


### 60 sampai 90

In [22]:
df_all_before = pd.read_csv('data_clean_reduce_21_z_score_3_sec.csv')
df_all = df_all_before[(df_all_before["α"] >= 60) & (df_all_before["α"] <= 90)]
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [23]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999997531009 
 Beta : 0.8334210184637281 
 V : 0.9999988329249242 
MAE : 
 Alpha : 3.867342634645877e-05 
 Beta : 1.5074948473187613 
 V : 0.006169856816769337 
MSE : 
 Alpha : 2.4344714284679252e-08 
 Beta : 7.970546013152887 
 V : 0.00022555947729439028 
Max AERR : 
 V     0.101429
α     0.001643
β    15.860507
dtype: float64
Waktu yang dibutuhkan :  0.6121227661768596
